In [10]:
import json
import fitz
import pandas as pd
import os
from langchain.docstore.document import Document
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map  # or thread_map
from multiprocessing import cpu_count

from multiprocessing import Manager
from functools import partial
import gc

In [2]:
!kaggle datasets download -d Cornell-University/arxiv

100%|██████████████████████████████████████| 1.17G/1.17G [01:53<00:00, 11.1MB/s]
100%|██████████████████████████████████████| 1.17G/1.17G [01:53<00:00, 11.2MB/s]


In [2]:
# Get a list of dicts and convert into a pandas df.
arxiv_data = []
for line in open('../arxiv-metadata-oai-snapshot.json', 'r'):
    arxiv_data.append(json.loads(line))
df = pd.DataFrame.from_records(arxiv_data)

In [3]:
df_filtered = df[df['categories'].str.contains('cs', na=False)]

In [6]:
len(df_filtered)/len(df)

0.3264255406434721

In [4]:
len(df_filtered)

749538

In [7]:
del arxiv_data
gc.collect()
del df
gc.collect()

22

In [3]:
basedir="/home/soma4/YEONDOO-arxiv-with-version/YEONDOO-arxiv/data/"
year="14"
month="01"
path_data=os.path.join(basedir,year,month)
id_list=os.listdir(path_data)

In [4]:
id_list=[id.split('v')[0] for id in os.listdir(path_data)]

In [18]:
id_list[0]

'1401.4606'

In [153]:
df[df['id']==id_list[2]]['categories'].item().split(' ')

['cs.AI', 'cs.LO']

In [154]:
df[df['id']==id_list[2]]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
493844,1401.4613,Peter Jeavons,"Peter Jeavons, Justyna Petke",Local Consistency and SAT-Solvers,None,"Journal Of Artificial Intelligence Research, V...",10.1613/jair.3531,None,cs.AI cs.LO,http://arxiv.org/licenses/nonexclusive-distrib...,Local consistency techniques such as k-consi...,"[{'version': 'v1', 'created': 'Sat, 18 Jan 201...",2014-01-21,"[[Jeavons, Peter, ], [Petke, Justyna, ]]"


In [146]:
id="1401.0001"

In [138]:
df[df['id']==id]['update_date']

493837    2014-01-21
Name: update_date, dtype: object

In [132]:
papper_id=id
published=df[df['id']==id]['update_date']
title=df[df['id']==id].title.item()

authors_replaced=df[df['id']==id].authors.item().replace('and',',').split(',')
authors=authors_replaced.split(',')
summary=df[df['id']==id].abstract.item()
url="http://arxiv.org/abs/"+id
ref=df[df['id']==id]['journal-ref'].item()

In [148]:
df[df['id']==id].authors.item().replace('and',',').split(',')

['Nils Bertschinger ',
 ' David H. Wolpert ',
 ' Eckehard Olbrich ',
 '\n  Juergen Jost']

In [147]:
df[df['id']==id].authors.item()

'Nils Bertschinger and David H. Wolpert and Eckehard Olbrich and\n  Juergen Jost'

In [141]:
published=df[df['id']==id]['update_date'].item()
published


'2014-01-21'

In [17]:
pdf_list[0][:-6]

'1401.4606'

In [19]:

def Wrapper(shared_list,doc_file_name):
    
    id=doc_file_name.split('v')[0]
    file_path=os.path.join(basedir,year,month,doc_file_name)
    try:
        with fitz.open(file_path) as doc_file:
            text: str = "".join(page.get_text() for page in doc_file)
    except:
        print(file_path)
        return
    result=df_filtered[df_filtered["id"]==id]

    metadata = {
        "Published": str(result["update_date"].item()),
        "Title": result.title.item(),
        "Authors": result.authors.item().replace('and',',').split(','),
        "Summary": result.abstract.item(),

        "paper_id": id,

        "journal_ref": result['journal-ref'].item(),
        
        "categories": result.categories,
        "source": "http://arxiv.org/abs/"+id,
    }
    doc = Document(
        page_content=text, metadata=metadata
    )
    shared_list.append(doc)

In [ ]:

def Wrapper2(shared_list,inputs):
    
    doc_file_name = inputs[0]
    result=inputs[1]
    id=doc_file_name.split('v')[0]
    file_path=os.path.join(basedir,year,month,doc_file_name)
    with fitz.open(file_path) as doc_file:
        text: str = "".join(page.get_text() for page in doc_file)
    
    # result=df[df["id"]==id]

    metadata = {
        "Published": str(result["update_date"].item()),
        "Title": result.title.item(),
        "Authors": result.authors.item().replace('and',',').split(','),
        "Summary": result.abstract.item(),

        "paper_id": id,

        "journal_ref": result['journal-ref'].item(),
        
        "categories": result.categories,
        "source": "http://arxiv.org/abs/"+id,
    }
    doc = Document(
        page_content=text, metadata=metadata
    )
    shared_list.append(doc)

In [9]:
basedir="/home/soma4/YEONDOO-arxiv-with-version/YEONDOO-arxiv/data/"
year="14"
month="01"
path_data=os.path.join(basedir,year,month)
pdf_list=os.listdir(path_data)

In [13]:
inputs=[]
for pdf_id in tqdm(pdf_list):
    tmp=[]
    tmp.append(pdf_id)
    id=pdf_id.split('v')[0]
    tmp.append(df_filtered[df_filtered["id"]==id])
    inputs.append(tmp)

100%|██████████| 202/202 [00:16<00:00, 12.24it/s]


In [25]:

chunksize=2

manager = Manager()
shared_list = manager.list()
process_map(partial(Wrapper, shared_list),pdf_list,max_workers=5,chunksize=chunksize)

  0%|          | 0/202 [00:00<?, ?it/s]

/home/soma4/YEONDOO-arxiv-with-version/YEONDOO-arxiv/data/14/01/filelist.txt


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [21]:
shared_list[0]

Document(page_content='Journal of Artiﬁcial Intelligence Research 42 (2011) 607-659\nSubmitted 09/11; published 12/11\nDrake: An Eﬃcient Executive for Temporal\nPlans with Choice\nPatrick R. Conrad\nprconrad@mit.edu\nBrian C. Williams\nwilliams@mit.edu\nRoom 32-227\n32 Vassar St\nCambridge, MA 02139 USA\nAbstract\nThis work presents Drake, a dynamic executive for temporal plans with choice. Dy-\nnamic plan execution strategies allow an autonomous agent to react quickly to unfolding\nevents, improving the robustness of the agent. Prior work developed methods for dynami-\ncally dispatching Simple Temporal Networks, and further research enriched the expressive-\nness of the plans executives could handle, including discrete choices, which are the focus of\nthis work. However, in some approaches to date, these additional choices induce signiﬁcant\nstorage or latency requirements to make ﬂexible execution possible.\nDrake is designed to leverage the low latency made possible by a preprocessi